In [2]:
from robotic import ry
import numpy as np
import time

print('ry version', ry.__version__, ry.compiled())
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

ry version 0.0.19 compile time: Sep 23 2023 14:58:54
-- ry.cpp:operator():99(0) python,
physx/motorKp: 10000,
physx/motorKd: 1000


In [3]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('push_cargo.g')
C.addFile('push_maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

In [3]:
C.view()

0

In [4]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'cargo'], ry.OT.eq, [1e2])
komo.addObjective([2], ry.FS.positionDiff, ['base', 'cargo'], ry.OT.eq, [1e2])
komo.addObjective([3], ry.FS.positionDiff, ['base', 'cargo'], ry.OT.eq, [1e2])
komo.addObjective([4], ry.FS.positionDiff, ['base', 'cargo'], ry.OT.eq, [1e2])
komo.addObjective([5], ry.FS.positionDiff, ['base', 'cargo'], ry.OT.eq, [1e2])
komo.addObjective([6], ry.FS.positionDiff, ['base', 'cargo'], ry.OT.eq, [1e2])

ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


{ time: 0.001283, evals: 7, done: 1, feasible: 0, sos: 2.05799, f: 0, ineq: 0, eq: 192.04 }
size of path: (6, 3)
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):65523.1 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    44281.2  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    27722.9  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    15848.1  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     8656.8  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):       6149  ACCEPT
--newton-- it:   6  |Delta|: 0.00709004  alpha:          1  evals:   7  f(y):    6146.06  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:   7  A(x):    6146.06  f:    2.05799  g:          0  h:     192.04  |x-x'

0

In [5]:
#define a path finding problem
rrt = ry.PathFinder()
i = 0
while True:
    i = i + 1
    print(i)
    rrt.setProblem(C, [qHome], [qT])
    ret = rrt.solve()
    path = ret.x
    print("rettttt: ", ret)
    #print("path size:", path)
    # display the path
    if hasattr(ret, 'x') and isinstance(ret.x, np.ndarray) and ret.x.ndim == 2 and ret.x.shape[0] > 0:
        for t in range(0, path.shape[0] - 1):
            C.setJointState(path[t])
            time.sleep(0.002)
        break

1
rettttt:  { time: 0.000478, evals: 17, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [6]:
C.setJointState(path[0])
C.view()

13

In [7]:
bot = ry.BotOp(C, False)
bot.home(C)

    
for i in range(0, 600):
    i= i*100
    bot.moveTo(path[i])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

IndexError: index 100 is out of bounds for axis 0 with size 22